<a href="https://colab.research.google.com/github/jorgejunior618/EstudosMachineLearning/blob/main/07_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
import pandas as pd
import numpy as np

dados = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/alura-clustering-validation/base-de-dados/CC%20GENERAL.csv')
dados = dados.drop(columns=['CUST_ID'])

In [160]:
def pipe(funcoes):
  def aplicar(valorInicial):
    resultado = valorInicial
    for funcao in funcoes:
      resultado = funcao(resultado)
    return resultado
  return aplicar

In [161]:
from sklearn.preprocessing import Normalizer

def normalizarColunas(data):
  normalizador = Normalizer()
  normalizador.set_output(transform="pandas")

  return normalizador.fit_transform(data)

def removeConstantes(data):
  '''
  Função que trata o conjunto de dados a ser usado no treinamento do modelo
  de aprendizagem de maquina, de forma a utilizar somente dados relevantes
  para a obtenção de um resultado preciso.
_____________________________________
  Remove as colunas que possuam uma variancia muito baixa, ou seja,
  que seu valor é praticamente o mesmo em todas as ocorrências
  '''
  dataCols = data.columns
  remover = []
  for col in dataCols:
    if len(dados[col].value_counts()) < 10:
      remover.append(col)
  return data.drop(columns=remover)

def trataValoresNulos(data):
  nulos = data.isna().sum()
  return data.fillna(dados.median())

def preProcessarDados(data):
  return pipe([
    removeConstantes,
    trataValoresNulos,
    # normalizarColunas
  ])(data)

In [162]:
dados = preProcessarDados(dados)
dados

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT
0,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000
1,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222
2,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000
3,1666.670542,0.636364,1499.00,1499.00,0.00,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,312.343947,0.000000
4,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8945,28.493517,1.000000,291.12,0.00,291.12,0.000000,1.000000,0.000000,0.833333,0.000000,0,6,1000.0,325.594462,48.886365,0.500000
8946,19.183215,1.000000,300.00,0.00,300.00,0.000000,1.000000,0.000000,0.833333,0.000000,0,6,1000.0,275.861322,312.343947,0.000000
8947,23.398673,0.833333,144.40,0.00,144.40,0.000000,0.833333,0.000000,0.666667,0.000000,0,5,1000.0,81.270775,82.418369,0.250000
8948,13.457564,0.833333,0.00,0.00,0.00,36.558778,0.000000,0.000000,0.000000,0.166667,2,0,500.0,52.549959,55.755628,0.250000


# Algorítmo de Clusterização K-Means (K-Médias)

Um dos mais simples e conhecidos algoritmos de divisão de clusters para aprendizado não supervisionado

## Funcionamento

- Define K pontos (centroides) aleatórios iniciais para a clusterização;
- Mede a distância dos centróides de cada item da base de dados
- Seleciona os itens mais proximos e divide nos clusters
- Recalcula os centróides com base na divisão feita
- Repete o processo de clusterização com base nas novas distâncias

In [163]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, n_init=10, max_iter=300)

y_pred = kmeans.fit_predict(normalizarColunas(dados))

print(y_pred)
print(len(y_pred))

[1 3 1 ... 1 1 0]
8950


# Metricas de Validação Internas

Mede a compactação dos clusters e a distância enter os clusters como forma de validação da boa divisão feita pelo algorítmo

## Coeficiente de Silhouette


```math
S = (b - a)/max(a,b)
```
Onde:
- a -> Distancia média entre cada ponto dentro de um mesmo cluster (compactação)
- b -> Distancia média das menores distancias dos pontos dos clusters entre si (distância)
- S -> O coeficiente de silhouette

O valor de *S* varia de -1 à 1, de forma que quanto mais proximo de 1 for o coeficiente, melhor foi a clusterização, e quanto mais próxmo de -1, pior


In [164]:
from sklearn.metrics import silhouette_score

S_score = silhouette_score(dados, y_pred, metric='euclidean')

S_score

0.07088508361154766

## Coef. Davies-Bouldin

semelhante ao metodo do Coefi. de Silhouette, porém com a base dos calculos feitas a partir do centroide de cada um dos clusters

In [165]:
from sklearn.metrics import davies_bouldin_score

DB_score = davies_bouldin_score(dados, y_pred)

DB_score

2.238133490612335

In [166]:
from sklearn.metrics import calinski_harabasz_score

CH_score = calinski_harabasz_score(dados, y_pred)

CH_score

473.5434880255218

# Validação Relativa

## Validadndo com base de dados aleátoria
Gerando um metodo que generalize a criação de clusters, é possivel realizar testes criando bases de dados aleatórias, de forma que se deseja verificar que para as mesmas configurações de clusterização a avaliação deva ser muito melhor para a base de dados real.

In [167]:
def clusterizar(n_clusters, data):
  kmeans = KMeans(n_clusters=n_clusters, n_init=10, max_iter=300)
  norm_data = normalizarColunas(data)
  labels = kmeans.fit_predict(norm_data)

  S_score = silhouette_score(norm_data, labels, metric='euclidean')
  DB_score = davies_bouldin_score(norm_data, labels)
  CH_score = calinski_harabasz_score(norm_data, labels)

  return S_score, DB_score, CH_score, labels, kmeans.cluster_centers_

In [168]:
dados_s1, dados_db1, dados_ch1, _, _ = clusterizar(3, dados)
dados_s2, dados_db2, dados_ch2, labels_dados, centroides_dados = clusterizar(5, dados)
dados_s3, dados_db3, dados_ch3, _, _ = clusterizar(10, dados)

In [169]:
dados_teste = np.random.rand(8950, 16)
teste_s2, teste_db2, teste_ch2, _, _ = clusterizar(5, dados_teste)

print("\ntam. cluster dados  5: ", dados_s2, dados_db2, dados_ch2)
print("             teste  5: ", teste_s2, teste_db2, teste_ch2)


tam. cluster dados  5:  0.36454479258047573 1.0760464906093192 3431.790347716922
             teste  5:  0.041575492648003934 3.402097018542553 319.51326645630405


## Validadndo com subdivisões da Base de Dados

Este método tem como objetivo verificar quea clusterização realizada manterá um padrão de avaliação aproximadamente constante para as mesmas configurações, porém utilizando partições menores desta Base de dados

In [170]:
set1, set2, set3 = np.array_split(dados, 3)

particao_s1, particao_db1, particao_ch1, _, _ = clusterizar(5, set1)
particao_s2, particao_db2, particao_ch2, _, _ = clusterizar(5, set2)
particao_s3, particao_db3, particao_ch3, _, _ = clusterizar(5, set3)

print(dados_s1, dados_db1, dados_ch1)
print(particao_s1, particao_db1, particao_ch1)
print(particao_s2, particao_db2, particao_ch2)
print(particao_s3, particao_db3, particao_ch3)

0.3271718523559296 1.310409951916838 3526.454518249131
0.3686777986013522 1.0591332190701965 1204.060255623223
0.35416642754504835 1.1382306445993162 1194.951986504888
0.3617082519102253 1.1530065355003551 1159.0102864676853


# Interpretação dos Clusters Obtidos

In [193]:
_, _, _, labels_dados, centroides_dados = clusterizar(5, dados)

qtd_attr = len(centroides_dados[0])

remover_cols = []
for i in range(qtd_attr):
  if centroides_dados[:,i].var() < 0.01:
    remover_cols.append(dados.columns.values[i])
remover_cols.append('MINIMUM_PAYMENTS')
remover_cols.remove('PRC_FULL_PAYMENT')

dados_analise = dados.drop(columns=remover_cols)
dados_analise['cluster'] = labels_dados

In [194]:
desc_dados = dados_analise.groupby("cluster")

n_clients = desc_dados.size()
desc_dados = desc_dados.mean()
desc_dados['n_clients'] = n_clients

desc_dados

,BALANCE,PURCHASES,CASH_ADVANCE,CREDIT_LIMIT,PAYMENTS,PRC_FULL_PAYMENT,n_clients
cluster,,,,,,,
0,443.746756,629.249107,141.482978,5130.547795,814.294226,0.246806,3280
1,1794.443291,478.411620,3266.686592,3983.399639,4712.044614,0.117321,1074
2,3037.962543,385.248630,1636.917210,4495.771989,968.890376,0.001799,2649
3,1142.514535,3274.126719,181.727235,4096.718849,3036.609179,0.278331,1536
4,1987.501586,854.865815,421.129352,2227.737226,1336.238911,0.019318,411


In [173]:
# Visualizando os dados no plano e plotando os clusters

# from sklearn.manifold import TSNE

# tsne = TSNE(n_components=2)

# valores_plano = tsne.fit_transform(dados)

In [174]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# sns.scatterplot(
#   x=valores_plano[:,0],
#   y=valores_plano[:,1],
#   hue=y_pred,
#   palette={
#       0: 'red',
#       1: 'pink',
#       2: 'black',
#       3: 'green',
#       4: 'yellow',
#   }
# )